In [24]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from time import time

In [2]:
! wget https://datasets.imdbws.com/name.basics.tsv.gz 
! wget https://datasets.imdbws.com/title.ratings.tsv.gz
! wget https://datasets.imdbws.com/title.akas.tsv.gz
! wget https://datasets.imdbws.com/title.basics.tsv.gz
! wget https://datasets.imdbws.com/title.crew.tsv.gz
! wget https://datasets.imdbws.com/title.episode.tsv.gz
! wget https://datasets.imdbws.com/title.principals.tsv.gz
! wget https://datasets.imdbws.com/title.episode.tsv.gz

--2020-08-28 11:14:50--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.33.242.82, 13.33.242.54, 13.33.242.62, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.33.242.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202568893 (193M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.5’

name.basics.tsv.gz. 100%[===================>] 193,18M  4,43MB/s    in 44s     

2020-08-28 11:15:35 (4,36 MB/s) - ‘name.basics.tsv.gz.5’ saved [202568893/202568893]

--2020-08-28 11:15:35--  https://datasets.imdbws.com/title.ratings.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.33.242.82, 13.33.242.54, 13.33.242.62, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.33.242.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5281119 (5,0M) [binary/octet-stream]
Saving to: ‘title.ratings.tsv.gz.3’

title.ratings.tsv.g 100%[===================>]   5,

# Choose imdb-ids of most popular movies

In [147]:
fpath = "./title.ratings.tsv.gz"

df_ratings = pd.read_table(fpath)
# print(df_ratings.head())

df = pd.read_table("./title.basics.tsv.gz", na_values={"startYear": ["\\N"], "endYear": ["\\N"], "isAdult": ["\\N"]})
# print(df.head())

df = df.merge(df_ratings, on="tconst")

#fill start year values
df["startYear"] = df["startYear"].fillna(value=df["startYear"])
# print(df["startYear"].isna().any())

df["startYear"] = df["startYear"].fillna(value=0)
# print(df["startYear"].isna().any())

df["endYear"] = df["endYear"].fillna(value=df["startYear"])
# print(df["endYear"].isna().any())

df["isAdult"] = df["isAdult"].fillna(value=0)
# print(df["isAdult"].isna().any())

df = df.astype(dtype={"startYear": np.int32, 
                 "endYear": np.int32, 
                 "isAdult": np.int32})
# print(df.head())
# print(df["titleType"].unique())

In [148]:
target = ["movie", "tvMovie", "tvSeries", "tvMiniSeries"] 
ind_drop = df[~df['titleType'].isin(target)].index
df = df.drop(ind_drop)
print(df.shape)

(572990, 11)


In [149]:
collect_movies_based_on_rating = False
collect_movies_based_on_numvotes = True

if collect_movies_based_on_rating:

    movies_ids = []

    movies_ids.extend(df.loc[(df["startYear"] <= 1990)  & (df["averageRating"] > 8), "tconst"].values)
    print(len(movies_ids))

    movies_ids.extend(df.loc[(df["startYear"] > 1990) & (df["startYear"] <= 2005) & (df["averageRating"] > 7), "tconst"].values)
    print(len(movies_ids))

    movies_ids.extend(df.loc[(df["startYear"] > 2005) & (df["startYear"] <= 2015) & (df["averageRating"] > 7), "tconst"].values)
    print(len(movies_ids))

    movies_ids.extend(df.loc[(df["startYear"] > 2015) & (df["startYear"] <= 2020) & (df["averageRating"] > 6), "tconst"].values)
    print(len(movies_ids))

    with open("all_imdb_ids.txt", "w") as f:
        for movie_id in movies_ids:
            f.write(str(movie_id) + "\n")

if collect_movies_based_on_numvotes:

    movies_ids = []

    movies_ids.extend(df.loc[df.loc[:, "numVotes"] > 1000, "tconst"].values)
    print(len(movies_ids))

    movies_ids.extend(df.loc[df.loc[:, "averageRating"] > 8., "tconst"].values)
    print(len(movies_ids))
    
    movies_ids = list(set(movies_ids))
    print(f"Total length of the database: {len(movies_ids)}")

    with open("most_popular_imdb_ids.txt", "w") as f:
        for movie_id in movies_ids:
            f.write(str(movie_id) + "\n")

60300
113162
Total length of the database: 69470


In [150]:
with open("most_popular_imdb_ids.txt", "r") as f:
    all_movies_ids = f.read().splitlines()
    
all_movies_ids = list(set(all_movies_ids))
print(f"Total number of considered movies: {len(all_movies_ids)}")

Total number of considered movies: 69470


# Collect titles and ratings

In [156]:
t0 = time()
fpath = "./title.ratings.tsv.gz"
df_ratings = pd.read_table(fpath)

ind_drop = df_ratings[~df_ratings['tconst'].isin(all_movies_ids)].index
df_ratings = df_ratings.drop(ind_drop)
assert df_ratings.shape[0] == len(all_movies_ids), print("Number of samples less than number of movies")
# print(df_ratings.head())


fpath = "./title.basics.tsv.gz"

df = pd.read_table(fpath, na_values={"startYear": ["\\N"], "endYear": ["\\N"], "isAdult": ["\\N"]})
# print(df.head())

ind_drop = df[~df['tconst'].isin(all_movies_ids)].index
df = df.drop(ind_drop)

df = df.merge(df_ratings, on="tconst")

df.rename(columns={"originalTitle": "original title",
                   "primaryTitle": "title",
                   "genres": "genre",
                   "averageRating": "imdb_rating",
                   "tconst": "imdb_id"
                  }, inplace=True)

# print(df.head())
df.drop_duplicates(inplace=True)
# print(df.shape)

df["titleType"] = df["titleType"].apply(lambda x: "Series" if "Series" in x else "")
df["genre"] = [",".join([x,y]) if y != "" else x for x,y in zip(df["genre"], df["titleType"])]
df["genre"] = df["genre"].apply(lambda x: x if x != "\\N" else "")
df["genre"] = df["genre"].apply(lambda x: x.split(","))

df.fillna({"startYear": 0, "endYear": 0}, inplace=True)
df["startYear"] = df["startYear"].astype("int")
df["endYear"] = df["endYear"].astype("int")
df.drop(["titleType", "isAdult", "runtimeMinutes"], axis=1, inplace=True)
assert df.shape[0] == len(all_movies_ids), print("Number of samples less than number of movies")

print(f"Total time: {time() - t0}")

Total time: 15.78906536102295


# Collect names of actors etc

In [157]:
t0 = time()
fpath = "./title.principals.tsv.gz"

df_principals = pd.read_table(fpath)
df_principals = df_principals.loc[:, ["tconst", "nconst", "ordering", "category"]]
df_principals.rename(columns={"tconst": "imdb_id"}, inplace=True)

ind_drop = df_principals[~df_principals['imdb_id'].isin(all_movies_ids)].index
df_principals = df_principals.drop(ind_drop)

ind_drop = df_principals[~df_principals['ordering'].isin([1, 2, 3, 4, 5, 6])].index
df_principals = df_principals.drop(ind_drop)

df_principals["category"] = df_principals["category"].apply(lambda x: x if x != "actress" else "actor")
target_profs = ["director", "producer", "actor", "writer"] 
ind_drop = df_principals[~df_principals['category'].isin(target_profs)].index
df_principals = df_principals.drop(ind_drop)
# print(df_principals.head())
    
fpath = "./name.basics.tsv.gz"

df_names = pd.read_table(fpath)
df_names = df_names.loc[:, ["primaryName", "nconst"]]
# print(df_names.head())

df_principals = df_principals.merge(df_names, on="nconst")

print(f"Total time: {time() - t0}")

Total time: 54.29540514945984


# Collect persons

In [158]:
t0 = time()

def collect_movie_persons(x):
     return pd.Series({f"{prof}s": x.loc[x.sort_values(by=["ordering"])["category"] == prof, "primaryName"].values
                       for prof in ["director", "producer", "actor", "writer"]})
    
df_principals = pd.DataFrame(df_principals.groupby('imdb_id').apply(collect_movie_persons))

df.set_index("imdb_id", inplace=True)
df = df.join(df_principals, on="imdb_id")
df.reset_index(inplace=True)

df.fillna(value={f"{prof}s": "" for prof in target_profs}, inplace=True)

assert df.shape[0] == len(all_movies_ids), print("Number of samples less than number of movies")

print(f"Total time: {time() - t0}")

Total time: 228.78745293617249


# Collect alternative titles

In [159]:
t0 = time()

fpath = "./title.akas.tsv.gz"

df_akas = pd.read_table(fpath)
# df_akas = df_akas.loc[:, ["tconst", "nconst", "ordering", "category"]]
df_akas = df_akas.loc[df_akas["region"] == "US", :]
df_akas.rename(columns={"titleId": "imdb_id"}, inplace=True)

ind_drop = df_akas[~df_akas['imdb_id'].isin(all_movies_ids)].index
df_akas = df_akas.drop(ind_drop)
print(df_akas.shape)
print(df_akas.head())

grouped_data = df_akas.groupby('imdb_id')['title'].apply(lambda x: '::'.join(x))
df_titles = pd.DataFrame(grouped_data)
df_titles.rename(columns={"title": "all_titles"}, inplace=True)
# df_titles.set_index("imdb_id", inplace=True)

df.set_index("imdb_id", inplace=True)
df = df.join(df_titles, on="imdb_id")
df.reset_index(inplace=True)

df.fillna(value={"all_titles": ""}, inplace=True)

assert df.shape[0] == len(all_movies_ids), print("Number of samples less than number of movies")

print(f"Total time: {time() - t0}")

(48623, 8)
        imdb_id  ordering                                      title region  \
4840  tt0002130        12                            Dante's Inferno     US   
6233  tt0002844        10  Fantômas: In the Shadow of the Guillotine     US   
6250  tt0002844         6              Fantomas: The Beltham Mystery     US   
6603  tt0003037        11                 Fantomas: The Man in Black     US   
6606  tt0003037        14             Fantômas: Juve versus Fantômas     US   

     language        types    attributes isOriginalTitle  
4840       \N           \N            \N               0  
6233       \N  imdbDisplay            \N               0  
6250       \N           \N  review title               0  
6603       \N  imdbDisplay            \N               0  
6606       \N  imdbDisplay            \N               0  
Total time: 20.65169882774353


In [160]:
df.head()

,imdb_id,title,original title,startYear,endYear,genre,imdb_rating,numVotes,directors,producers,actors,writers,all_titles
0,tt0002130,Dante's Inferno,L'Inferno,1911,0,"[Adventure, Drama, Fantasy]",7.0,2118,"[Francesco Bertolini, Adolfo Padovan]",[],"[Salvatore Papa, Arturo Pirovano, Giuseppe de ...",[],Dante's Inferno
1,tt0002844,Fantômas: In the Shadow of the Guillotine,Fantômas - À l'ombre de la guillotine,1913,0,"[Crime, Drama]",7.0,1893,[Louis Feuillade],[],"[René Navarre, Edmund Breon, Georges Melchior,...",[Marcel Allain],Fantômas: In the Shadow of the Guillotine::Fan...
2,tt0003037,Fantomas: The Man in Black,Juve contre Fantômas,1913,0,"[Crime, Drama]",7.0,1320,[Louis Feuillade],[],"[René Navarre, Edmund Breon, Georges Melchior,...",[Marcel Allain],Fantomas: The Man in Black::Fantômas: Juve ver...
3,tt0003165,Fantômas: The Dead Man Who Killed,Le mort qui tue,1913,0,"[Crime, Drama, Mystery]",7.0,1020,[Louis Feuillade],[],"[René Navarre, Edmund Breon, Georges Melchior,...",[Marcel Allain],Fantômas: The Dead Man Who Killed
4,tt0003419,The Student of Prague,Der Student von Prag,1913,0,"[Drama, Fantasy, Horror]",6.5,1707,[Stellan Rye],[],"[Paul Wegener, Grete Berger, Lyda Salmonova, J...",[Hanns Heinz Ewers],A Bargain with Satan::The Student of Prague


In [162]:
database = df.to_dict("records")
for el in database:
    el["genre"] = el["genre"] if el["genre"] != "" else None
    el["startYear"] = el["startYear"] if el["startYear"] != 0 else None
    el["endYear"] = el["endYear"] if el["endYear"] != 0 else None
    el["all_titles"] = el["all_titles"].split("::") if el["all_titles"] != "" else []
    for prof in ["director", "producer", "actor", "writer"]:
        el[f"{prof}s"] = list(el[f"{prof}s"]) if list(el[f"{prof}s"]) != "" else []

print(len(database))
with open("database_most_popular_main_info.json", "w") as f:
    json.dump(database, f, indent=2)

69470


In [163]:
database[100]

{'imdb_id': 'tt0015119',
 'title': 'The Marriage Circle',
 'original title': 'The Marriage Circle',
 'startYear': 1924,
 'endYear': None,
 'genre': ['Comedy'],
 'imdb_rating': 7.2,
 'numVotes': 1023,
 'directors': ['Ernst Lubitsch'],
 'producers': [],
 'actors': ['Florence Vidor', 'Monte Blue', 'Marie Prevost', 'Creighton Hale'],
 'writers': ['Paul Bern'],
 'all_titles': ['The Marriage Circle']}